# Segmenting and Clustering Neighborhoods in Toronto - Part 2 :

## Add the latitude and the longitude coordinates of each neighborhood

### Imports and Initialization

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Now assign the link of the website through which we are going to scrape the data and assign it to variable named website_url, and create a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [2]:
# getting data from aspecific source (wikipedia) 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Find the required table are
soup = BeautifulSoup(website_url,'lxml')

# extracting the raw table inside that webpage
table = soup.find('table', {'class': 'wikitable sortable'})

### Data processing 1 :  Extract the table to a base dataframe (from Part 1)
The base dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
table_pr = table.find_all('tr')

# Extract the table header (Extract three columns PostalCode, Borough, and Neighborhood)
table_header_pr = table_pr[0].findAll('th')
table_header = [str(th).replace('<th>', '').replace('</th>', '').replace('\n', '') for th in table_header_pr]

# Extract the table body
table_body_pr = table_pr[1:]
table_body = [list(map(lambda td: str(td if not td.findAll("a") 
                                         else td.find("a").getText()).replace('<td>', '')
                          .replace('</td>', '').replace('\n', '')
                          , tr.findAll('td'))) for tr in table_body_pr]


# Create a base dataframe to clean and transform
df_base = pd.DataFrame(table_body, columns=table_header)

### Data processing 2 :  Ignore cells with a borough that is not assigned (from Part 1)

In [4]:
# replace "Not assigned" to NaN : To use dropna function for dropping the cells that have an Not assigned borough
df_base.replace("Not assigned", np.nan, inplace=True)

# Drop cells with "Not assigned" borough 
df_base.dropna(subset=["Borough"], inplace=True)

### Data processing 3 :  Replace Not assigned neighborhood by the borough (from Part 1)
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df_base.Neighbourhood.fillna(df_base.Borough, inplace=True)

### Data processing 4 :  Combine neighborhood that have the same postal code into one row (from Part 1)
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
# Rename Postcode column to PostalCode to be used in the final Dataframe
df_base.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

# Combine neighboorhood Strings that have the same postal code to one row 
df_toronto_part1 = df_base.groupby(['PostalCode', "Borough"])[["Neighbourhood"]].agg(lambda x: ', '.join(x)).reset_index()

###  Data processing 4 -  Extracting Latitude and Longtitude for each Postal code in Toronto (New)

Using the csv file proposed in the course to create the dataframe with the geographical coordinates for each postal code

In [7]:
!wget -q -O "geographical_coordinates.csv" http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [8]:
df_coordinates = pd.read_csv('geographical_coordinates.csv')
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [9]:
# merge the datafarame of toronto neighborhoods with toronto postal code coordinates(Latitude and Longtitude)
df_toronto = df_toronto_part1.join(df_coordinates.set_index('PostalCode'), on='PostalCode')

### Final result : Visualize the appropriate pandas Dataframe - with Latitude & Longitude

In [10]:
# Visualizing the result Dataframe
df_toronto.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#####  <span style="color:red">Information :</span> The Dataframe is displayed in an ordered form 